In [1]:
import kanji_lists
import pandas as pd
import re

#My Stuff
from importlib import reload  
import minnie
import front
import uniform
import parse

In [24]:
soup = minnie.scrape('https://www.google.com/maps/place/Target/data=!4m5!3m4!1s0x8822e954c79dade1:0xc4fc7bd7807d9d62!8m2!3d42.7350682!4d-84.4822379?authuser=0&hl=en&rclk=1',fancy=True)

# len(str(a[1]))
# i=0
# for e in a:
#     print(len(str(e)),i)
#     i+=1
    
#a[11]
soup

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/Place" lang="en"> <head> <link href="/maps/preview/opensearch.xml?hl=en" rel="search" title="Google Maps" type="application/opensearchdescription+xml"/> <title>  Google Maps  </title> <meta content=" Find local businesses, view maps and get driving directions in Google Maps. " name="Description"/> <meta content="initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/> <meta content="chrome=1" http-equiv="X-UA-Compatible"/> <meta content="notranslate" name="google"/> <meta content="origin" name="referrer"/> <meta content="ByHT0GXztW_RcGxS0o86DBf1WtNu02FfqlcT8njnSqU" name="google-site-verification"/> <meta content="Diln__r3p9-tt39P2Cl2Amvx6oFB4PATnxuFBaw6ej8" name="google-site-verification"/> <meta content="Target · 201 E Grand River Rd, East Lansing, MI 48823" itemprop="name"/> <meta content="Target · 201 E Grand River Rd, East Lansing, MI 48823" property="og:title"/> <meta content="https:

In [2]:
#Throw together a big list of strings
asa_links = front.asahi()
nhk_links = front.nhk()
yom_links = front.yomiuri()
mai_links = front.mainichi()
print(f"""Link count for today:\nAsahi: {len(asa_links)}\nNHK: {
len(nhk_links)}\nYomiuri: {len(yom_links)}\nMainichi: {len(mai_links)}""")

Link count for today:
Asahi: 101
NHK: 9
Yomiuri: 102
Mainichi: 80


In [ ]:
#To prevent reading the same article
#Couldn't you also just run every week?
#There won't be any singular same article from a week ago.

#OOO! Idea. If you tried to keep a permanent list it would get to big to compare and check such long strings
#But if you cleaned it every couple of days 
#it wouldn't be too taxing to compare a couple hundred 30+ character strings to one another

def link_check(asa,nhk,yom,mai):
    """Removes articles that have already been scraped and parsed."""
    
    print('Lol')
    return (asa,nhk,yom,mai)

asa_links,nhk_links,yom_links,mai_links = link_check(asa_links,nhk_links,yom_links,mai_links)

In [ ]:
dai_gomi = []

i=0
for l in asa_links:
    gomi = uniform.asahi(l)
    if gomi == None:
        print(f"Timed out on run {i}, moving on...")
    else:
        dai_gomi.append(gomi)
    i+=1
    print(f"{i} out of {len(asa_links)}")
    
i=0
for l in nhk_links:
    gomi = uniform.nhk(l)
    dai_gomi.append(gomi)
    i+=1
    print(f"{i} out of {len(nhk_links)}")

i=0
for l in yom_links:
    gomi = uniform.yomiuri(l)
    dai_gomi.append(gomi)
    i+=1
    print(f"{i} out of {len(yom_links)}")

i=0
for l in mai_links:
    gomi = uniform.mainichi(l)
    dai_gomi.append(gomi)
    i+=1
    print(f"{i} out of {len(mai_links)}")
    
#To avoid index error in the jukugo algorithm
#We add a final character so the: @@@while gomi[i+1] in KANJI@@@ can finish
for i in range(len(dai_gomi)):
    dai_gomi[i]=dai_gomi[i]+'。'

In [ ]:
#Perform parsing

listy=[]
prefixs=[]
suffixs=[]
i=0
for g in dai_gomi:
    metrics,listy,prefixs,suffixs = parse.jukugo(g,listy,prefixs,suffixs)
    print(metrics,i)
    i+=1
        
print(len(suffixs),len(prefixs),len(listy))
juku = pd.Series(listy).value_counts()
pre = pd.Series(prefixs).value_counts()
suf = pd.Series(suffixs).value_counts()

#Ive noticed this algorithm will pick up names.
#Names are from 3 to 5 characters usually.
#It's the fourcharacter names that cause trouble for this algorithm though. 
#(three and five character names cause 田 to be one of the most common suffixs lol)
#They get split equally with no question.
#And then you get names coming up in the vocab.
#That's fine and all but I wonder what we can do about it.

In [ ]:
#Drop mistakes, clean the data. It's slow

import jisho
#juku[juku.index[0]]
print(len(juku))
i=0
for j in juku.index:
    counts = jisho.kanji_tag_count(j)
    if counts['Common word'] == 0:
        juku.pop(j)
    i+=1
    print(i,len(juku))
    
#Cleaning it after the fact is very slow. Unfortunately it's slow because of the html request, not the processing.
#We need to figure out how to query the JMdict. It is THE resource used for Jisho.org and basically any other 英和辞典

In [ ]:
def count_distr(listy):
    """Function to help make sense of the distribution of character/word counts in an immense list"""
    
    old = listy[0]
    i=0
    for l in listy:
        if l == old:
            i+=1
            old = l
        else:        
            print(f"Count Number {old}, Number of those counts: {i}")
            old = l
            i = 1
            
count_distr(suf)

In [ ]:
#You're gonna have to think of a way to write everyday. (Read things in, append, write out)
#Then when the code is ran again, to cross reference already read articles, so only new ones are added.
#When added you also need to think about how consolidate a new set of results.
#Do you want to just keep some long as list? Pull it in and do a value_counts on it?
#Or do you wanna value counts each time and add the results together? (First option seems easier tbh)

def update(juku,pre,suf):
    load_juku = pd.read_csv('juku.csv',index_col=0)
    load_pre = pd.read_csv('prefixs.csv',index_col=0)
    load_suf = pd.read_csv('suffixs.csv',index_col=0)


    for j in juku.index:
        try:
            load_juku.loc[j] = load_juku.loc[j] + juku[j]
        except KeyError:
            load_juku.loc[j] = juku[j] 

    for p in pre.index:
        try:
            load_pre.loc[j] = load_pre.loc[j] + pre[j]
        except KeyError:
            load_pre.loc[j] = pre[j] 

    for s in suf.index:
        try:
            load_suf.loc[j] = load_suf.loc[j] + suf[j]
        except KeyError:
            load_suf.loc[j] = suf[j] 

    load_juku.to_csv('juku.csv')
    load_pre.to_csv('prefixs.csv')
    load_suf.to_csv('suffixs.csv')